In [28]:
import os
import re
from tika import parser

In [29]:
##### Scraping Don Quijote

pdf_dir = '/Users/eun-youngchristinapark/Documents/UBC-CAPSTONE/pdf_scraping/'
donquijote = 'Recorridos_Don-Quijote-Student-text-2017.pdf'
file_name = pdf_dir + donquijote

In [41]:
#### Get the raw text
raw = parser.from_file(file_name)
raw_list = raw['content'].strip().split('\n\nRecorridos 3—Siglo XVI     Cervantes ')  ## contains pages of the book
text_list = []
regex = r'[0-9]+\. '
regex2 = r'[0-9]+\-'
regex3 = r'Capítulo [0-9]+  \n\n'
regex4 = r'[0-9]+\.'
regex5 = r'\n+\s+[0-9]+\s+\n+'

# extract text only - remove questions / discussions / vocabularies etc. 
# text begins at page 12 and ends at page 85

for i in range(11,84):
    if 'BANCO DE PALABRAS ÚTILES' not in raw_list[i]:  # skip End-of-Chapter Questions 
        
        ### Get the text boundary
        
        if re.search(regex3, raw_list[i]) == None:
            begin_index = raw_list[i].find('\n\n')
            end_index = raw_list[i][:raw_list[i].find('=')+1].rfind('\n\n')
            end_index_prev = 0 
            if raw_list[i][begin_index:begin_index+30].startswith('\n\n¿COMPRENDISTE LO QUE LEÍSTE?'):            # skip the end of chapter question section 
                for match in re.finditer(regex4, raw_list[i]):
                    begin_index = match.span()[0]
                begin_index = raw_list[i].find('\n\n', begin_index + 1)
                end_index = raw_list[i][begin_index+1:raw_list[i].find('=')+1].rfind('\n\n') + begin_index+1
                
            elif begin_index == end_index:                                                                        # the page starts with vocabulary list: we have to skip this section
                while end_index != -1:
                    end_index_prev += end_index + 1
                    end_index = raw_list[i][end_index_prev+1:].find('=')
                begin_index = raw_list[i].find('\n\n', end_index_prev + 1)
                end_index = 1
                search_begin_index = begin_index 
                while end_index > 0:
                    regex_match = re.search(regex5, raw_list[i][search_begin_index + 1:])
                    if regex_match:
                        end_index = regex_match.span()[0]
                        search_begin_index += end_index 
                    else:
                        end_index = -1 
                end_index = search_begin_index
        else:
            begin_index = re.search(regex3, raw_list[i]).span()[0]
            end_index = raw_list[i][begin_index+1:raw_list[i].find('=')+1].rfind('\n\n') + begin_index+1
        
        
        # For each sentence, determine whether the sentence is in the text 
        
        ts = raw_list[i][begin_index:end_index]   # main text 
        ts_list = ts.split('\n')
        ts_list_filtered = []
        qu_list = [0] * len(ts_list)
        for idx, sent in enumerate(ts_list):
            qu_list[idx] = 0
            if sent.strip() == '' and idx > 0:                                                                        # empty strings 
                qu_list[idx] = qu_list[idx-1]
            elif sent.strip().startswith('¿') or re.match(regex, sent.strip()) or re.match(regex2, sent.strip()):     # textbook questions or comments
                if sent.strip().endswith('.') or sent.strip().endswith('?') or re.match(regex2, sent.strip()):
                    qu_list[idx] = 1                                                                                  # question started and ended
                else:
                    qu_list[idx] = 2                                                                                  # question started (not ended)
            elif idx > 0 and (qu_list[idx-1] == 2 or qu_list[idx-1] == 3):
                if sent.strip().endswith('.') or sent.strip().endswith('?'):
                    qu_list[idx] = 4                                                                                  # question ended
                else:
                    qu_list[idx] = 3                                                                                  # question continues 
    
            elif sent.strip().startswith('“'):
                if sent.strip().endswith('”'):
                    qu_list[idx] = 5                                                                                  # quotation started and ended
                else:
                    qu_list[idx] = 6                                                                                  # quotation started (not ended)
            elif idx > 0 and qu_list[idx-1] == 6 and not(sent.strip().endswith('”')):
                qu_list[idx] = 7                                                                                      # quotatoin continues
            elif idx > 0 and (qu_list[idx-1] == 6 or qu_list[idx-1] == 7) and sent.strip().endswith('”'):
                qu_list[idx] = 8                                                                                      # quotation ended 
            if idx > 0 and qu_list[idx-1] == 8:
                qu_list[idx] = 9
        
            if qu_list[idx] == 0:
                ts_list_filtered.append(sent)
        ts_filtered = '\n'.join(ts_list_filtered)
        text_list.append(ts_filtered)

In [23]:
chap_div = []
for i in range(len(text_list)):
    if 'Capítulo' in (text_list[i][:100]):
        print(i)
        print(text_list[i][:100])
        chap_div.append(i)

0
Capítulo I 
Que trata de la condición y ejercicio del famoso
hi dalgo don Quijote de la Mancha  
 de
5
Capítulo 2  
 ¡Oh, cómo se holgó nuestro buen caballero cuando hubo 
hecho este discurso, y más cuan
6
Capítulo II 
Que trata de la primera salida que de su tierra hizo el    
ingenioso don Quijote  
 He
31
Capítulo VI y VII     (RESUMEN) 
Del donoso y gran escrutinio que el cura y el barbero 
hicieron en 


In [39]:
i = 54
text_list[i]

' Tuvo a todo el mundo en poco, \nfue el espantajo y el coco del \nmundo, en tal coyuntura, que \nacreditó su ventura morir cuerdo \ny vivir loco.  \n Cervantes añadió al final de su obra: \n —Para mí sola nació don Quijote, y yo para él; él supo obrar y \nyo escribir, solos los dos somos para en uno, a despecho y pesar del \nescritor fingido y tordesillesco que se atrevió o se ha de atrever a \nescribir con pluma de avestruz grosera y mal deliñada las hazañas \nde mi valeroso caballero, porque no es carga de sus hombros, ni \nasunto de su resfriado ingenio; a quien advertirás, si acaso llegas \na conocerle, que deje reposar en la sepultura los cansados y ya \npodridos huesos de don Quijote, y no le quiera llevar, contra todos \nlos fueros de la muerte, a Castilla la Vieja, haciéndole salir de la \nfuesa donde real y verdaderamente yace tendido de largo a largo, \nimposibilitado de hacer tercera jornada y salida nueva: que para \nhacer burla de tantas como hicieron tantos andantes caba

In [50]:
chpts = [0,6,13,19,26,31,42,49,51]

chap_list = []
chap_str = ''
for i in range(len(text_list)):
    if i in set(chpts):
        if i!= 0:
            chap_list.append(chap_str)
        chap_str = text_list[i]
    else:
        chap_str = chap_str + text_list[i]
chap_list.append(chap_str)

In [51]:
len(chap_list)

9

In [56]:
import json
SOURCE = 'https://espanolabierto.org/recorridos/'
AUTHOR = 'Cervantes'
TITLE = 'Recorridos_Don-Quijote-Student-text-2017'
LEVEL = 'A2'
dict_list = []

for i in range(len(chap_list)):
    story_data = {}
    story_data['source'] = SOURCE
    story_data['author'] = AUTHOR
    story_data['title'] = TITLE
    story_data['level'] = LEVEL
    story_data['content'] = chap_list[i]
    dict_list.append(story_data)
    
with open('a2_donquijote.txt', 'w') as outfile:
        json.dump(dict_list, outfile)

In [57]:
with open('a2_donquijote.txt') as json_file:
    data = json.load(json_file)
    
print(type(data))
print(len(data))
print(type(data[0]))
print(data[0], '\n')
print(data[0]['content'])

<class 'list'>
9
<class 'dict'>
{'source': 'https://espanolabierto.org/recorridos/', 'author': 'Cervantes', 'title': 'Recorridos_Don-Quijote-Student-text-2017', 'level': 'A2', 'content': 'Capítulo I \nQue trata de la condición y ejercicio del famoso\nhi dalgo don Quijote de la Mancha  \n de cuyo nombre no \nquiero acordarme, no ha mucho tiempo que vivía un hidalgo \nde los de lanza en astillero, adarga antigua, rocín flaco y galgo \ncorredor. Una olla de algo más vaca que carnero, salpicón las \nmás noches, duelos y quebrantos los sábados, lentejas los \nviernes, algún palomino de añadidura los domingos, consumían \nlas tres partes de su hacienda. El resto della concluían sayo \nde velarte, calzas de velludo para las fiestas, con sus pantu-\nflos de lo mesmo, y los días de entresemana se honraba consu \nvellorí de lo más fino. Tenía en su casa una ama que pasaba  \nde los cuarenta, y una sobrina que no llegaba a los veinte, y \nun mozo de campo y plaza, que así ensillaba el rocín como 